In [22]:
import pandas as pd
import numpy as np
from random import sample
from pandas import *
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Libraries from Alan
from __future__ import division
import json
import os
from matplotlib.pyplot import *

import requests
#import tmy_ids

# Show Plots in the Notebook
%matplotlib inline

# Make Plots larger
rcParams['figure.figsize']= (10, 6)
rcParams['font.size'] = 14

%matplotlib inline

print 'Pandas version: ' + pd.__version__

Pandas version: 0.14.0


In [23]:
tmy3_to_noaa = [
    (702730, 'Anchorage Intl AP', 'II', 'ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US'),
    (703980, 'Annette Island AP', 'II',  'ANNETTE WEATHER SERVICE OFFICE AIRPORT AK US'),
    (700260, 'Barrow W Post-W Rogers Arpt [NSA - ARM]', 'II', 'BARROW W POST W ROGERS AIRPORT AK US'),
    (702190, 'Bethel Airport', 'II', 'BETHEL AIRPORT AK US'),
    (701740, 'Bettles Field', 'II', 'BETTLES AIRPORT AK US'),
    (702670, 'Big Delta Allen AAF', 'II', 'BIG DELTA AIRPORT AK US'),
    (703160, 'Cold Bay Arpt', 'II', 'COLD BAY AIRPORT AK US'),
    (702610, 'Fairbanks Intl Arpt', 'II', 'FAIRBANKS INTERNATIONAL AIRPORT AK US'),
    (702710, 'Gulkana Intermediate Field', 'II',  'GULKANA AIRPORT AK US'),
    (703410, 'Homer Arpt', 'II', 'HOMER AIRPORT AK US'),
    (703810, 'Juneau Intl Arpt', 'II', 'JUNEAU INTERNATIONAL AIRPORT AK US'),
    (703260, 'King Salmon Arpt', 'II', 'KING SALMON AIRPORT AK US'),
    (703500, 'Kodiak Airport', 'II',  'KODIAK AIRPORT AK US'),
    (701330, 'Kotzebue Ralph Wein Memorial', 'II', 'KOTZEBUE RALPH WEIN MEMORIAL AIRPORT AK '),
    (702310, 'McGrath Arpt', 'II', 'MCGRATH AIRPORT AK US'),
    (702000, 'Nome Municipal Arpt', 'II', 'NOME MUNICIPAL AIRPORT AK US'),
    (703080, 'St Paul Island Arpt', 'II', 'ST PAUL ISLAND AIRPORT AK US'),
    (702510, 'Talkeetna State Arpt', 'II', 'TALKEETNA AIRPORT AK US'),
    (703610, 'Yakutat State Arpt', 'II', 'YAKUTAT AIRPORT AK US'),
    (702986, 'Big River Lake', 'II', 'BIG RIVER LAKES AK US'),
    (702960, 'Cordova', 'II','CORDOVA M K SMITH AIRPORT AK US'),
    (704890, 'Dutch Harbor', 'II', 'DUTCH HARBOR AK US'),
    (702650, 'Fairbanks/Eielson A', 'II', 'EIELSON FIELD AK US'),
    
    #'Homer Arpt' is 7 miles away from Homer 8 NW.  The airport is on the coast and the NW site is inland.
    # Auke bay:  do we need this one?
    # Alyeska: don't think we need it
    # Cannery: middle of nowhere
    # College 5 NW, College, North Pole, University Experiment Station are unnecessary since we have Fairbanks station
    # Cooper landing: no nearby Tmy3
    # Eagle airport - nothing nearby
    # Glen alps : no station
    # Kitoi bay : nothing nearby, on Kodiak island
    # Kuparuk : 28 miles from deadhorse - use deadhorse tmy station?
    # little port walter not near anything
    # Matanuska Agricultural : 6 miles from Palmer Municipal TMY3 station
    (702740, 'Palmer Municipal', 'II', 'MATANUSKA AGRICULTURAL EXPERIMENT STATION AK US'),
    # main bay: not near anything
    # Tok : nothing nearby
    # White's crossing : no nearby tmy3
    # Nabesna: nothing close
    # Sutton: 10 miles from Palmer Municipal Airport tmy3 station
    # Port Alsworth: Nothing nearby
    (703400, 'Iliamna Arpt', 'II', 'ILIAMNA AIRPORT AK US'),
    (702590, 'Kenai Municipal AP', 'II', 'KENAI MUNICIPAL AIRPORT AK US'),
    (702910, 'Northway Airport', 'II', 'NORTHWAY AIRPORT AK US'),
    (702770, 'Seward', 'II', 'SEWARD AIRPORT AK US'),
    (703710, 'Sitka Japonski AP', 'II', 'SITKA AIRPORT AK US'),
    (701780, 'Tanana Ralph M Calhoun Mem AP', 'II', 'TANANA CALHOUN MEMORIAL AIRPORT AK US'),
    (702750, 'Valdez Wso', 'II', 'VALDEZ WEATHER SERVICE OFFICE AK US'),
    (703870, 'Wrangell', 'II', 'WRANGELL AIRPORT AK US')
]

In [24]:
tmy_ids = DataFrame(tmy3_to_noaa, columns=['ids', 'tmy_station', 'tmy_class', 'noaa_index'])
tmy_ids.count()

ids            32
tmy_station    32
tmy_class      32
noaa_index     32
dtype: int64

In [25]:
final_solar = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\final_solar_sites.csv")
final_solar.head()

,Unnamed: 0,stn_id,stn_name,ghi_uncert,dni_uncert,tmy_type,tmy_class,south_vert_solar,elevation,latitude,longitude,label
0,3,702730,Anchorage Intl AP,0.110,0.138,3,II,2.43,35,61.182999,-150.000000,2.43-0.14
1,7,703980,Annette Island AP,0.108,0.124,3,II,2.15,33,55.049999,-131.567001,2.15-0.12
2,9,700260,Barrow W Post-W Rogers Arpt [NSA - ARM],0.105,0.092,3,II,1.98,10,71.320000,-156.619995,1.98-0.09
3,10,702190,Bethel Airport,0.113,0.130,3,II,2.52,38,60.783001,-161.832993,2.52-0.13
4,11,701740,Bettles Field,0.113,0.137,3,II,2.73,196,66.917000,-151.516998,2.73-0.14


In [26]:
subset = final_solar[['stn_id', 'stn_name', 'tmy_class']]
final_solar_tuple = [tuple(x) for x in subset.values]
final_solar_tuple

[(702730L, 'Anchorage Intl AP', 'II'),
 (703980L, 'Annette Island AP', 'II'),
 (700260L, 'Barrow W Post-W Rogers Arpt [NSA - ARM]', 'II'),
 (702190L, 'Bethel Airport', 'II'),
 (701740L, 'Bettles Field', 'II'),
 (702670L, 'Big Delta Allen AAF', 'II'),
 (703160L, 'Cold Bay Arpt', 'II'),
 (702610L, 'Fairbanks Intl Arpt', 'II'),
 (702710L, 'Gulkana Intermediate Field', 'II'),
 (703410L, 'Homer Arpt', 'II'),
 (703810L, 'Juneau Intl Arpt', 'II'),
 (703260L, 'King Salmon Arpt', 'II'),
 (703500L, 'Kodiak Airport', 'II'),
 (701330L, 'Kotzebue Ralph Wein Memorial', 'II'),
 (702310L, 'McGrath Arpt', 'II'),
 (702000L, 'Nome Municipal Arpt', 'II'),
 (703080L, 'St Paul Island Arpt', 'II'),
 (702510L, 'Talkeetna State Arpt', 'II'),
 (702750L, 'Valdez Wso', 'II'),
 (703610L, 'Yakutat State Arpt', 'II')]

In [27]:
# PVWatts Helper functions.  PVWatts is used to calculate vertical south-facing solar.

def pvw_inputs():
    '''Returns a default set of PVWatts inputs.
    Note that there are other inputs documented here:  http://developer.nrel.gov/docs/solar/pvwatts-v5/
    For example, the 'timeframe' input will allow you to get back hourly results if you want.
    '''
    # substitute your NREL API Key here.  Dustin's key:  4kV70P6KKoeySaf4ZCEmEtQRZt7ngjrPtzo0zUKB 
    # See: http://developer.nrel.gov/docs/api-key/
    p = {'api_key': '4kV70P6KKoeySaf4ZCEmEtQRZt7ngjrPtzo0zUKB'}
    p['system_capacity'] = 1.0
    p['module_type'] = 0
    p['losses'] =  14
    p['array_type'] = 0
    p['azimuth'] =  180.0
    p['tilt'] = 48
    return p

def pvwatts(inputs):
    '''Run PVWatts and return the results as a Python dictionary.
    '''
    res = json.loads(requests.get('http://developer.nrel.gov/api/pvwatts/v5.json', params=inputs).text)
    # separate out just the outputs of the run and return that as the first element of the tuple.
    # The second return item is the full results dictionary which includes the inputs as well.
    return res['outputs'], res

In [28]:
def calc_solar(file_id, azimuth):
    '''Runs PVWatts for south facing vertical using the climate data in 
    the climate file with id 'file_id'.  Returns key results in a dictionary.
    '''
    inputs = pvw_inputs()
    inputs['file_id'] = file_id
    inputs['tilt'] = 90    # get vertical solar
    inputs['azimuth'] = azimuth
    outs, all = pvwatts(inputs)
    results = {'latitude': all['station_info']['lat'], 
               'longitude': all['station_info']['lon'],
               'elevation': all['station_info']['elev'],
               'south_vert_solar': round(outs['solrad_annual'], 2)
              }
    return results
    

In [29]:
'''Runs PVWatts for eacah compass direction using the climate data in the 
climate file with id 'file_id'.  Returns key results in a dictionary.
'''
def compass_vert_solar(file_id):
    compass_directions = [0.0, 90.0, 180.0, 270.0]
    inputs = pvw_inputs()
    inputs['file_id'] = file_id
    inputs['tilt'] = 90 #get vertical solar
    outs, all = pvwatts(inputs)
    results = {'latitude': all['station_info']['lat'], 
               'longitude': all['station_info']['lon'],
               'elevation': all['station_info']['elev'],
               'month': [1,2,3,4,5,6,7,8,9,10,11,12]
               }
    
    # Get monthly solar radiation for all 4 cardinal compass directions.
    for direction in compass_directions:
               inputs['azimuth'] = direction
               outs, all = pvwatts(inputs)
               column_name = 'azimuth - %d' % direction
               results[column_name] = outs['solrad_monthly']
                
    #get horizontal solar
    inputs['tilt'] = 0
    outs, all = pvwatts(inputs)
    results['Horizontal_Solar'] = outs['solrad_monthly']
    
    # Format as DF
    results = pd.DataFrame(results)
    return results
        

In [30]:
# Check horizontal solar calcs
#inputs = pvw_inputs()
#inputs['file_id'] = '1-702730'
#inputs['tilt'] = 0
#outs, all = pvwatts(inputs)
#results = outs['solrad_monthly']
#results = pd.DataFrame(results)
#inputs['azimuth'] = 0
#outs, all = pvwatts(inputs)
#results['North'] = outs['solrad_monthly']
#results

In [31]:
# Calculate the solar values for each site and then append it to a dataframe
outputs = pd.DataFrame()

for ids, site, tmy_class in final_solar_tuple:
    res = compass_vert_solar('1-%s' % ids)
    res['tmy_type'] = 3
    res['tmy_class'] =  tmy_class
    res['stn_id'] = ids
    res['stn_name'] = site
    outputs = outputs.append(res)

outputs.head()            

,Horizontal_Solar,azimuth - 0,azimuth - 180,azimuth - 270,azimuth - 90,elevation,latitude,longitude,month,tmy_type,tmy_class,stn_id,stn_name
0,0.286398,0.129053,0.767704,0.221557,0.229549,35,61.182999,-150,1,3,II,702730,Anchorage Intl AP
1,1.073078,0.367618,2.639147,0.945464,0.897813,35,61.182999,-150,2,3,II,702730,Anchorage Intl AP
2,2.224414,0.726061,3.110145,1.678331,1.588174,35,61.182999,-150,3,3,II,702730,Anchorage Intl AP
3,3.527861,1.119363,3.533270,2.357856,2.488320,35,61.182999,-150,4,3,II,702730,Anchorage Intl AP
4,4.483374,1.637356,3.239341,2.818602,2.931602,35,61.182999,-150,5,3,II,702730,Anchorage Intl AP


In [32]:
# Convert from kWh/m2/day to Btu/ft2/day
outputs['East_Vert_Solar'] = outputs['azimuth - 90'] * 317.1
outputs['North_Vert_Solar'] = outputs['azimuth - 0'] * 317.1
outputs['West_Vert_Solar'] = outputs['azimuth - 270'] * 317.1
outputs['South_Vert_Solar'] = outputs['azimuth - 180'] * 317.1
outputs['HorizSol'] = outputs['Horizontal_Solar'] * 317.1
outputs = outputs.drop(['azimuth - 0','azimuth - 90','azimuth - 180','azimuth - 270', 'Horizontal_Solar'], 1)

# Reorder the columns
outputs = outputs.reindex_axis(['stn_id', 'stn_name', 'month', 'tmy_type', 'tmy_class', 'South_Vert_Solar','East_Vert_Solar', 'West_Vert_Solar','North_Vert_Solar','HorizSol' , 'elevation', 'latitude', 'longitude'], axis=1)

outputs.head()

,stn_id,stn_name,month,tmy_type,tmy_class,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,elevation,latitude,longitude
0,702730,Anchorage Intl AP,1,3,II,243.439017,72.790127,70.255741,40.922696,90.816917,35,61.182999,-150
1,702730,Anchorage Intl AP,2,3,II,836.873376,284.696415,299.806488,116.571676,340.273159,35,61.182999,-150
2,702730,Anchorage Intl AP,3,3,II,986.226933,503.609820,532.198766,230.233863,705.361790,35,61.182999,-150
3,702730,Anchorage Intl AP,4,3,II,1120.400031,789.046156,747.676149,354.949878,1118.684836,35,61.182999,-150
4,702730,Anchorage Intl AP,5,3,II,1027.195038,929.611070,893.778646,519.205526,1421.677782,35,61.182999,-150


In [33]:
solar_df = outputs.merge(tmy_ids, left_on='stn_id', right_on='ids', how='left')
solar_df.head()

,stn_id,stn_name,month,tmy_type,tmy_class_x,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,elevation,latitude,longitude,ids,tmy_station,tmy_class_y,noaa_index
0,702730,Anchorage Intl AP,1,3,II,243.439017,72.790127,70.255741,40.922696,90.816917,35,61.182999,-150,702730,Anchorage Intl AP,II,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
1,702730,Anchorage Intl AP,2,3,II,836.873376,284.696415,299.806488,116.571676,340.273159,35,61.182999,-150,702730,Anchorage Intl AP,II,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
2,702730,Anchorage Intl AP,3,3,II,986.226933,503.609820,532.198766,230.233863,705.361790,35,61.182999,-150,702730,Anchorage Intl AP,II,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
3,702730,Anchorage Intl AP,4,3,II,1120.400031,789.046156,747.676149,354.949878,1118.684836,35,61.182999,-150,702730,Anchorage Intl AP,II,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
4,702730,Anchorage Intl AP,5,3,II,1027.195038,929.611070,893.778646,519.205526,1421.677782,35,61.182999,-150,702730,Anchorage Intl AP,II,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US


In [34]:
solar = solar_df[['noaa_index', 'month', 'South_Vert_Solar', 'East_Vert_Solar',
 'West_Vert_Solar','North_Vert_Solar','HorizSol']]
solar = solar.rename(columns = {'noaa_index' : 'station_name'})
solar.head()

,station_name,month,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1,243.439017,72.790127,70.255741,40.922696,90.816917
1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,2,836.873376,284.696415,299.806488,116.571676,340.273159
2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,3,986.226933,503.609820,532.198766,230.233863,705.361790
3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,4,1120.400031,789.046156,747.676149,354.949878,1118.684836
4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,5,1027.195038,929.611070,893.778646,519.205526,1421.677782


In [35]:
solar.to_csv("20_Solar_Sites_For_AKWarm.csv")